In [89]:
import pandas as pd
import pyodbc
import sqlalchemy.types
from icecream import ic
from sqlalchemy import create_engine
from urllib import parse
from timeit import default_timer as timer

In [27]:
# ODBC Connection for FileMaker
start = timer()
cnxn = pyodbc.connect('DSN=FM Clamp ODBC;UID=FMODBC;PWD=FMODBC')
cnxn.timeout = 60
cursor = cnxn.cursor()
print("FileMaker Connect Time = " + str(round((timer() - start), 3)) + " sec")

FileMaker Connect Time = 9.501 sec


In [81]:
# SQLAlchemy connection for SQL Server
server = 'tn-sql'
database = 'autodata'
driver = 'ODBC+Driver+17+for+SQL+Server'
user = 'production'
pwd = parse.quote_plus("Auto@matics")
port = '1433'
database_conn = f'mssql+pyodbc://{user}:{pwd}@{server}:{port}/{database}?driver={driver}'
# Make Connection
engine = create_engine(database_conn)
# conn = engine.raw_connection()
conn = engine.connect()

In [4]:
# ODBC Connection for AS400
CONNAS400 = """
Driver={iSeries Access ODBC Driver};
system=10.143.12.10;
Server=AS400;
Database=PROD;
UID=SMY;
PWD=SMY;
"""

In [5]:
dbcnxn = pyodbc.connect(CONNAS400)
cursor = dbcnxn.cursor()
dbase = []

In [20]:
strsql = """SELECT PROD.FPSPRMAST1.SPH_PART,
                       STRIP(PROD.FPSPRMAST1.SPH_ENGPRT),
                       STRIP(PROD.FPSPRMAST1.SPH_DESC1),
                       STRIP(PROD.FPSPRMAST1.SPH_DESC2),
                       STRIP(PROD.FPSPRMAST1.SPH_MFG),
                       STRIP(PROD.FPSPRMAST1.SPH_MFGPRT),
                       STRIP(PROD.FPSPRMAST2.SPD_CABINT),
                       STRIP(PROD.FPSPRMAST2.SPD_DRAWER),
                       PROD.FPSPRMAST2.SPD_QOHCUR,
                       PROD.FPSPRMAST1.SPH_CURSTD,
                       STRIP(PROD.FPSPRMAST2.SPD_REODTE),
                       STRIP(PROD.FPSPRMAST2.SPD_USECC),
                       STRIP(PROD.FPSPRMAST2.SPD_PURCC),
                       STRIP(PROD.FPSPRMAST2.SPD_QREORD)
                FROM PROD.FPSPRMAST1 INNER JOIN PROD.FPSPRMAST2 ON PROD.FPSPRMAST1.SPH_PART = PROD.FPSPRMAST2.SPD_PART
                WHERE (((PROD.FPSPRMAST2.SPD_FACIL)=9))
"""

In [25]:
sql = """
            SELECT Ourpart,"Band A Part Number", "Housing A Part Number",
                "Screw Part Number" AS Screw, "Band Feed from Band data",
                "Ship Diam Max", "Ship Diam Min", "Hex Size", "Band_Thickness", "Band_Width",
                "CameraInspectionRequired", "ScrDrvChk"
            FROM tbl8Tridon 
"""

In [21]:
cursor.execute(strsql)
result_spares = cursor.fetchall()

In [22]:
data_type_dict = {'StandardCost' : float, 'OnHand' : int, 'PartNum' : str, 'ReOrderPt' : int}
df = pd.DataFrame.from_records(result_spares)
df.columns = ['PartNum', 'EngPartNum', 'Desc1', 'Desc2', 'Mfg', 'MfgPn', 'Cabinet', 'Drawer', 'OnHand', 'StandardCost','ReOrderDate', 'DeptUse', 'DeptPurch', 'ReOrderPt']
df = df.dropna()
df = df.astype(data_type_dict)
df = df.convert_dtypes()
df.dtypes

PartNum         string[python]
EngPartNum      string[python]
Desc1           string[python]
Desc2           string[python]
Mfg             string[python]
MfgPn           string[python]
Cabinet         string[python]
Drawer          string[python]
OnHand                   Int32
StandardCost           Float64
ReOrderDate     string[python]
DeptUse         string[python]
DeptPurch       string[python]
ReOrderPt                Int32
dtype: object

In [47]:
# coding=utf-8
start = timer()
cursor.execute(sql)
result = cursor.fetchall()

In [49]:
ic(result)

ic| result: [('DO NOT TYPE HERE', '\r', None, None, None, None, None, None, None, None, 'NO', 'NO'),
             ('M714006102', '1042006', '2703025', '1213002', '3.800', None, 0.83, '5/16"', 0.022, 0.5, 'NO', 'NO'),
             ('M674029102', '1083029', '2367012', '1278001', '8.41', 0.0, 0.0, '5/16"', 0.025, 0.5, 'NO', 'NO'),
             ('M674047102', '1083047', '2367012', '1278001', '12.165', 0.0, 0.0, '5/16"', 0.025, 0.5, 'NO', 'NO'),
             ('M679064102', '2019064', '2367012', '1278001MC', '14.500', 0.0, 0.0, '5/16"', 0.025, 0.5, 'NO', 'NO'),
             ('M674088102', '1083088', '2367012', '1278001', '19.988', 0.0, 0.0, '5/16"', 0.025, 0.5, 'NO', 'NO'),
             ('M725104106', '2009104', '2367025', '1278001', '23.028', 0.0, 0.0, '5/16"', 0.025, 0.5, 'NO', 'NO'),
             ('M674038102', '1083038', '2367012', '1278001', '9.9799999999999986', 0.0, 0.0, '5/16"', 0.025, 0.5, 'NO', 'NO'),
             ('M487056102', '2001056', '2057003', '2215002', '13.500', 0.0, 0.0, 

KeyboardInterrupt: 

In [129]:

df = pd.DataFrame.from_records(result)
df.columns = ['PartNumber', 'Band', 'Housing', 'Screw', 'Feed', 'DiaMax', 'DiaMin', 'HexSz', 'BandThickness','BandWidth', 'CamInspect', 'ScrDrvChk']
data_type_dict = {'PartNumber' : str, 'Band' : str,'Housing' : str,'Screw' : str, 'Feed' : float, 'DiaMax' : float,'DiaMin' : float,'HexSz' : str,'BandThickness' : float, 'BandWidth' : float,'CamInspect' : str,'ScrDrvChk' : str}
# df['PartNumber'] = df['PartNumber'].str.replace('\xa0', '')
df = df[df.Feed != 'N/A']
df = df[1:]
df['CamInspect'] = df['CamInspect'].str.upper()
df['ScrDrvChk'] = df['ScrDrvChk'].str.upper()
df['Feed'] = df['Feed'].round(3)
df['BandWidth'] = df['BandWidth'].round(3)
df.fillna({'DiaMax' : 0.0}, inplace=True)
df.fillna({'DiaMin' : 0.0}, inplace=True)
df = df.dropna()
df = df.astype(data_type_dict)
df = df.convert_dtypes()
df['HexSz'] = df['HexSz'].replace(r'\n', '', regex=True)
#df.sample(n=20)
ic(repr(df))
# Display Duplicated Part Numbers
# ids = df['PartNumber']
# df[ids.isin(ids[ids.duplicated()])].sort_values('PartNumber')


ic| repr(df): ('       PartNumber     Band  Housing      Screw    Feed  DiaMax  DiaMin  '
               'HexSz  \\
              '
               '1      M714006102  1042006  2703025    1213002     3.8     0.0    0.83  '
               '5/16"   
              '
               '2      M674029102  1083029  2367012    1278001    8.41     0.0     0.0  '
               '5/16"   
              '
               '3      M674047102  1083047  2367012    1278001  12.165     0.0     0.0  '
               '5/16"   
              '
               '4      M679064102  2019064  2367012  1278001MC    14.5     0.0     0.0  '
               '5/16"   
              '
               '5      M674088102  1083088  2367012    1278001  19.988     0.0     0.0  '
               '5/16"   
              '
               '...           ...      ...      ...        ...     ...     ...     ...    '
               '...   
              '
               '12151  M665030006  2147030  2157003    2173001    8.41     0.0    

'       PartNumber     Band  Housing      Screw    Feed  DiaMax  DiaMin  HexSz  \\\n1      M714006102  1042006  2703025    1213002     3.8     0.0    0.83  5/16"   \n2      M674029102  1083029  2367012    1278001    8.41     0.0     0.0  5/16"   \n3      M674047102  1083047  2367012    1278001  12.165     0.0     0.0  5/16"   \n4      M679064102  2019064  2367012  1278001MC    14.5     0.0     0.0  5/16"   \n5      M674088102  1083088  2367012    1278001  19.988     0.0     0.0  5/16"   \n...           ...      ...      ...        ...     ...     ...     ...    ...   \n12151  M665030006  2147030  2157003    2173001    8.41     0.0     2.2  5/16"   \n12152   368004175  1081004  4097020  4208010MC   2.625     0.0    0.54   1/4"   \n12153   368010175  1081010  4097020  4208010MC   4.083     0.0     1.0   1/4"   \n12154   368016175  1081016  4097020  4208010MC   5.424     0.0    1.44   1/4"   \n12155  M368024102  1081024  4097020  4208010MC   6.994     0.0    1.44   1/4"   \n\n       BandT

In [23]:
df.to_sql('tblSpares', conn, schema='eng', if_exists='replace', index=False)

28

In [125]:
data_type_dict = {'PartNumber' : sqlalchemy.types.VARCHAR, 'Band' : sqlalchemy.types.VARCHAR,'Housing' : sqlalchemy.types.VARCHAR,'Screw' : sqlalchemy.types.VARCHAR, 'Feed' : sqlalchemy.types.Float, 'DiaMax' : sqlalchemy.types.Float,'DiaMin' : sqlalchemy.types.Float,'HexSz' : sqlalchemy.types.VARCHAR,'BandThickness' : sqlalchemy.types.Float, 'BandWidth' : sqlalchemy.types.Float,'CamInspect' : sqlalchemy.types.VARCHAR,'ScrDrvChk' : sqlalchemy.types.VARCHAR}

df.to_sql('parts_clamps', conn, schema='production', if_exists='replace', index=False, dtype=data_type_dict)

68

In [112]:
df.dtypes

PartNumber       string[python]
Band             string[python]
Housing          string[python]
Screw            string[python]
Feed                    Float64
DiaMax                  Float64
DiaMin                  Float64
HexSz            string[python]
BandThickness           Float64
BandWidth               Float64
CamInspect       string[python]
ScrDrvChk        string[python]
dtype: object

In [108]:
ic(data_type_dict)

ic| data_type_dict: {'Band': <class 'str'>,
                     'BandThickness': <class 'float'>,
                     'BandWidth': <class 'float'>,
                     'CamInspect': <class 'str'>,
                     'DiaMax': <class 'float'>,
                     'DiaMin': <class 'float'>,
                     'Feed': <class 'float'>,
                     'HexSz': <class 'str'>,
                     'Housing': <class 'str'>,
                     'PartNumber': <class 'object'>,
                     'ScrDrvChk': <class 'str'>,
                     'Screw': <class 'str'>}


{'PartNumber': object,
 'Band': str,
 'Housing': str,
 'Screw': str,
 'Feed': float,
 'DiaMax': float,
 'DiaMin': float,
 'HexSz': str,
 'BandThickness': float,
 'BandWidth': float,
 'CamInspect': str,
 'ScrDrvChk': str}